### 데이터 크롤링

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.request
import os
import time

def collect_image(search_word, extract_img_count):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors')
    options.add_argument('disable-infobars')
    options.add_argument('--disable-extensions')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')

    driver = webdriver.Chrome('chromedriver', options=options)
    driver.implicitly_wait(3)

    # 구글 이미지 검색 페이지 로드
    url = 'https://www.google.com/search?q=' + search_word + '&source=lnms&tbm=isch'
    driver.get(url)

    # 페이지 스크롤 다운
    for i in range(10):
        driver.execute_script("window.scrollBy(0, 10000);")
        time.sleep(1)

    # 이미지 추출
    imgs = driver.find_elements(By.CSS_SELECTOR, 'img.rg_i')

    # 추출된 이미지에서 src 속성값 추출
    img_srcs = []
    for img in imgs:
        src = img.get_attribute('src')
        if src and 'http' in src:
            img_srcs.append(src)

    # 이미지 다운로드
    if len(img_srcs) > extract_img_count:
        img_srcs = img_srcs[:extract_img_count]

    for i, src in enumerate(img_srcs):
        print('Downloading image', i+1)
        try:
            # 이미지 url 열기
            with urllib.request.urlopen(src) as url:
                # 이미지 파일 확장자 추출
                file_ext = os.path.splitext(src)[1]
                # 이미지 파일 저장
                with open(f"{search_word}_{i}{file_ext}", "wb") as f:
                    f.write(url.read())
        except Exception as e:
            print(e)

    driver.close()

if __name__ == '__main__':
    collect_image('Takase River', 150)


### 파일 끝에 확장자 붙이기

In [24]:
import os

directory = "/Users/nayoungmin/Desktop/Capstone/image_yeju/Ta"

for filename in os.listdir(directory):
    if not filename.endswith(".jpg"):
        src = os.path.join(directory, filename)
        dst = os.path.join(directory, filename + ".jpg")
        os.rename(src, dst)


### 네이버 데이터 크롤링

In [16]:
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time

driver = webdriver.Chrome()
driver.get("https://search.naver.com/search.naver?where=image&amp;sm=stb_nmr&amp;")
element = driver.find_element("id", "nx_query")

search_keyword = input("검색할 단어를 입력: ")
element.send_keys(search_keyword)
element.submit()

while True:
    driver.find_element("xpath", '//body').send_keys(Keys.END)
    time.sleep(10)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    imgList = soup.find_all("img", class_="_image _listImage")
    
    if len(imgList) >= 65:
        break

time.sleep(5)

def fetch_list_url():
    params = []
    imgList = soup.find_all("img", class_="_image _listImage")
    
    for im in imgList:
        try:
            params.append(im["data-lazy-src"])
            
        except:
            params.append(im["src"])

    return params

def fetch_detail_url():
    params = fetch_list_url()
    a = 1
    
    for p in params:
        urllib.request.urlretrieve(p, "/Users/nayoungmin/Desktop/Capstone/image_yeju/철로/" + search_keyword + str(a) + ".jpg")
        a += 1
        
fetch_detail_url()

driver.quit()


검색할 단어를 입력: 부산 미포철길
